In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py

In [ ]:
#MAIN DIRECTORIES
mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
scratchDirectory='/home/air673/koa_scratch/'
codeDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Variable_Calculation/CalculateMoreVariables'

In [ ]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str
    
def GetData(dataDirectory, parcelDirectory):
    dataNC = xr.open_dataset(dataDirectory, decode_timedelta=True) 
    parcelNC = xr.open_dataset(parcelDirectory, decode_timedelta=True) 
    return dataNC,parcelNC

def SubsetDataVars(dataNC):
    varList = ["thflux", "qvflux", "tsk", "cape", 
               "cin", "lcl", "lfc", "th",
               "prs", "rho", "qv", "qc",
               "qr", "qi", "qs","qg", 
               "buoyancy", "uinterp", "vinterp", "winterp",]
    
    varList += ["ptb_hadv", "ptb_vadv", "ptb_hidiff", "ptb_vidiff",
                "ptb_hturb", "ptb_vturb", "ptb_mp", "ptb_rdamp", 
                "ptb_rad", "ptb_div", "ptb_diss",]
    
    varList += ["qvb_hadv", "qvb_vadv", "qvb_hidiff", "qvb_vidiff", 
                "qvb_hturb", "qvb_vturb", "qvb_mp",]
    
    varList += ["wb_hadv", "wb_vadv", "wb_hidiff", "wb_vidiff",
                "wb_hturb", "wb_vturb", "wb_pgrad", "wb_rdamp", "wb_buoy",]

    return dataNC[varList]

[dataDirectory,parcelDirectory, res,t_res,Np_str,Nz_str] = GetDataDirectories(simulationNumber=1)
[data,parcel] = GetData(dataDirectory, parcelDirectory)

In [ ]:
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
if res=='1km':
    dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
if res=='250m':
    dir2='/mnt/lustre/koa/scratch/air673/'
def initiate_array():
    # Define array dimensions (adjust based on your netCDF)
    t_size = len(data['time'])  # Number of timesteps
    z_size = len(data['zh'])    # Number of vertical levels
    y_size = len(data['yh'])    # Number of y-axis points
    x_size = len(data['xh'])    # Number of x-axis points

    out_file = dir2 + 'Variable_Calculation/OUTPUT/' + f'Eulerian_VMF_{res}_{t_res}.h5'
    with h5py.File(out_file, 'a') as f:
        for var_name in ['VMF_c', 'VMF_g']:
            if var_name not in f:
                f.create_dataset(var_name, 
                                 (t_size, z_size, y_size, x_size), 
                                 maxshape=(None, z_size, y_size, x_size),
                                 dtype='float64', 
                                 chunks=(1, z_size, y_size, x_size))


            
def add_timestep_at_index(timestep_data_dict, index):
    out_file = dir2 + 'Variable_Calculation/OUTPUT/' + f'Eulerian_VMF_{res}_{t_res}.h5'
    with h5py.File(out_file, 'a') as f:
        if 'VMF_c' in timestep_data_dict:
            f['VMF_c'][index] = timestep_data_dict['VMF_c']
        if 'VMF_g' in timestep_data_dict:
            f['VMF_g'][index] = timestep_data_dict['VMF_g']


In [ ]:
def GetBinaryArray(t):
    ##############################################
    #READING BACK IN
    out_file2 = dir2 + 'Variable_Calculation/' + f'Eulerian_Binary_Array_{res}_{t_res}.h5'
    
    with h5py.File(out_file2, 'r') as f:
        A_c = f['A_c'][t]  # Loads the entire A_c array into memory
        A_g = f['A_g'][t]  # Loads the entire A_g array into memory
    return A_c,A_g

In [ ]:
####################################
#RUNNING

In [ ]:
w_thresh1=0.1
w_thresh2=0.5
qcqi_thresh=1e-6

In [ ]:
initiate_array()

#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in range(len(data['time'])):
    if np.mod(t,1)==0: print(f'Current time {t}')

    print('calculating variables')
    w_data=data['winterp'].isel(time=t).data
    rho_data=data['rho'].isel(time=t).data
    [A_c,A_g]=GetBinaryArray(t)
        
    VMF_c=rho_data*w_data*A_c
    VMF_g=rho_data*w_data*A_g

    print('outputting')
    result = {
        'VMF_c': VMF_c,
        'VMF_g': VMF_g
    }
    add_timestep_at_index(result, t)

In [ ]:
######################################################

In [ ]:
# ###############################################
# #READING BACK IN
# out_file = dir2 + 'Variable_Calculation/' + f'Eulerian_VMF_{res}_{t_res}.h5'

# with h5py.File(out_file, 'r') as f:
#     VMF_c = f['VMF_c'][t]  # Loads the entire A_c array into memory
#     VMF_g = f['VMF_g'][t]  # Loads the entire A_g array into memory

In [ ]:
# #####################
# #TESTING
# t=100
# z=10
# w_t=data['winterp'].isel(time=t,zh=z).data
# rho_t=data['rho'].isel(time=t,zh=z).data

# out_file = dir2 + 'Variable_Calculation/' + f'Eulerian_Binary_Array_{res}_{t_res}.h5'
# with h5py.File(out_file, 'r') as f:
#     A_c = f['A_c'][t,z]  # Loads the entire A_c array into memory
#     A_g = f['A_g'][t,z]  # Loads the entire A_g array into memory

# out_file = dir2 + 'Variable_Calculation/' + f'Eulerian_VMF_{res}_{t_res}.h5'
# with h5py.File(out_file, 'r') as f:
#     VMF_c = f['VMF_c'][t,z]  # Loads the entire VMF_c array into memory
#     VMF_g = f['VMF_g'][t,z]  # Loads the entire VMF_g array into memory

# print(np.all(rho_t*w_t*A_c==VMF_c))
# print(np.all(rho_t*w_t*A_g==VMF_g))